In [1]:
import pandas as pd
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from util.util import *
from features import *
import json
from scipy import stats

In [2]:
train_dir = '/home/i40/data/AGDTask2/train/'
train = [ train_dir + f for f in os.listdir(train_dir)]
train = sorted(train)
test_dir = '/home/i40/data/AGDTask2/test/'
test = [ test_dir + f for f in os.listdir(test_dir)]
test = sorted(test)
total = train + test

In [3]:
# Here are the codes of the CO-sensors and CO-alarms
sensors = ["CQ 32301 XQ01", "CQ 32302 XQ01", "CQ 32303 XQ01", "CQ 32304 XQ01", "CQ 32305 XQ01"]
alarms = ["CQ 32306 XH01", "CQ 32306 XH03", "CQ 32306 XH05"]

In [4]:
common_features = json.load(open("commonFeatures_new", 'r'))

In [5]:
train_features = common_features + sensors + alarms

In [6]:
def readFile(fn, train_features, alarms):
    temp = pd.read_csv(fn,engine='python' )
        # if alarm doe not exist, then add a correspond column
    for alarm in alarms:
        if alarm not in temp.columns:
            temp[alarm] = 0

    temp = temp[train_features]
    return temp

In [21]:
data = readFile(train[0],train_features,alarms)
data['time'] = pd.to_datetime(data['time'])
data = data.set_index('time')

In [23]:
X = data.drop(sensors+alarms, axis=1)

In [24]:
y = pd.DataFrame()
y['alarm'] = data["CQ 32306 XH01"]+data["CQ 32306 XH03"]+data["CQ 32306 XH05"]
y['alarm'] = (y['alarm']>0).astype(int)

In [27]:
f1 = X.resample('5min',how=np.mean)
f2 = X.resample('5min',how=np.std)
f3 = X.resample('5min',how=np.var)
X = pd.concat([f1,f2,f3],axis=1)

/home/i40/dbprak14/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """Entry point for launching an IPython kernel.
/home/i40/dbprak14/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  
/home/i40/dbprak14/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
y = y.resample('5min',how=np.max)

/home/i40/dbprak14/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """Entry point for launching an IPython kernel.


In [32]:
X = X[0:X.shape[0]-1]
y = y[1:y.shape[0]]

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=66)

In [39]:
print(X_train.shape,y_train.shape)

((258, 702), (258, 1))


In [40]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

/home/i40/dbprak14/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
sc = StandardScaler()
sc = sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

pca = PCA(n_components=0.95)
pca = pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [48]:
from sklearn.metrics import recall_score,precision_score
param_dist = {'n_estimators':100,'max_depth': 4}
bst = xgb.XGBClassifier(**param_dist)
bst = bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)

#result = 5*(recall*precision)/(4*precision+recall)

print('recall score: {}'.format(recall))
print('precision score {}'.format(precision))
#print('f2 score: {}'.format(result))


recall score: 0.0
precision score 0.0
